## Exercise 9.2

### Imports and dependencies

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import copy
import random
import sys

from sklearn import datasets as dsets
from sklearn.preprocessing import StandardScaler

from itertools import cycle, islice

### DP-Clustering

In [ ]:
# Function to compute the required sigma to achieve (<eps>, <delta>)-DP after <nb_iterations> iterations of the Gauss mechanism
def compute_gaussian_sigma(eps, nb_iterations, delta=1e-5):
	delta_term = 2 * (np.log(1.25)-np.log(delta))
	return np.sqrt(nb_iterations * delta_term) / eps

In [ ]:
# Clip the given value <val> at length <c>
def clip_len(val, c):
    ###### TODO (Part 1) ######
    # Implement  the clipping #
    ########## TODO ##########

In [ ]:
# DP-Clustering: Cluster the data points <x_given> into <k> clusters using <nb_iterations>. The initial centroids are <init_centroids>.
# To achieve DP, to be protected values are bounded in length by clipping bound <c_bound> and perturbed with noise scaled by <noise_multiplier>.
def dp_clustering(x_given, k, nb_iterations, eps, init_centroids, params):
    # Set initial centroids/cluster centers
    x = copy.deepcopy(x_given)
    centroids = copy.deepcopy(init_centroids)

    # Read given parameters
    c_bound = params['c_bound']
    noise_multiplier = compute_gaussian_sigma(eps, nb_iterations)
    
    # Calculate the distances between centroids and all data points
    distances = cdist(x, centroids, 'euclidean')
     
    # Label data points by nearest centroid
    labels = np.array([np.argmin(i) for i in distances])

    # Create storage for average minimum distances between centroids and their assigned data points
    avg_min_dist = []
     
    # For the defined number of iterations... 
    for _ in range(nb_iterations):
        # For all clusters...
        for idx in range(k):
            # Skip clusters with no data points assigned
            if len(x[labels==idx]) == 0:
                print("Info: Empty cluster created.")
                continue
            
            ########## TODO (Part 1) ##########
            # Implement the  centroid updates #
            ############## TODO  ##############
        
        # Calculate average minimum distance between centroids and datapoints after current iteration
        distances = cdist(x, centroids ,'euclidean')
        min_dist = np.array([np.amin(i) for i in distances])
        avg_min_dist.append(np.mean(min_dist))
        labels = np.array([np.argmin(i) for i in distances])
    
    # Return final data points' cluster labels, final centroids and final average minimum distance
    return labels, centroids, avg_min_dist

### ExpQ-Clustering

In [ ]:
def _u(d, i, m):
    return i + 1 - m if i < m else m - i


def _b_size(b):
    return np.abs( b[1] - b[0] )


def _exp_mechanism(d, B, m, eps):

    b_sizes = np.apply_along_axis( _b_size, 1, B )

    utilities = np.fromiter( (_u(d, i, m) for i in np.indices((B.shape[0],))[0]), np.float64 ) 
    
    weights = b_sizes * np.exp( eps * utilities )
    weights /= weights.sum()
    
    return weights


def _exp_q(d, eps):

    m = np.floor( d.shape[0] / 2 ).astype(int)  # Desired quantile -> Median

    B = np.array( [(x,y) for x,y in zip(d,d[1:])] )

    weights = _exp_mechanism( d, B, m, eps )

    indices = np.random.choice( np.indices((B.shape[0],))[0], p=weights, size=1 )
    bins = B[indices]

    unif_bin = lambda b: np.random.uniform(low=b[0], high=b[1])
    values = np.apply_along_axis( unif_bin, 1, bins )

    return values


def get_median(data, eps):
    """
    Note: <data> should be of the form [ [x1, y1, ...], [x2, y2, ...], ..., [xn, yn, ...] ]
    """

    data = np.sort( data.T, axis=1 )

    return np.array([ _exp_q(x, eps) for x in data ]).flatten()

In [ ]:
# ExpQ-Clustering: Use the median calculated using ExpQ to cluster the data points <x_given> into <k> clusters using <nb_iterations> iterations.
# The initial centroids are <init_centroids>.
def expq_clustering(x_given, k, nb_iterations, eps_target, init_centroids):

    eps = eps_target/nb_iterations

    # Set initial centroids/cluster centers
    x = copy.deepcopy(x_given)
    centroids = copy.deepcopy(init_centroids)

    # Calculate the distances between centroids and all data points
    distances = cdist(x, centroids, 'euclidean')
     
    # Label data points by nearest centroid
    labels = np.array([np.argmin(i) for i in distances])

    # Create storage for average minimum distances between centroids and their assigned data points
    avg_min_dist = []
     
    # For the defined number of iterations... 
    for _ in range(nb_iterations):
        # For all clusters...
        for idx in range(k):
            # Skip clusters with no data points assigned
            if len(x[labels==idx]) == 0:
                print("Info: Empty cluster created.")
                continue
            
            ########## TODO (Part 2) ##########
            # Implement the  centroid updates #
            ############## TODO  ##############
        
        # Calculate average minimum distance between centroids and datapoints after current iteration
        distances = cdist(x, centroids ,'euclidean')
        min_dist = np.array([np.amin(i) for i in distances])
        avg_min_dist.append(np.mean(min_dist))
        labels = np.array([np.argmin(i) for i in distances])
    
    # Return final data points' cluster labels, final centroids and final average minimum distance
    return labels, centroids, avg_min_dist

### Gather and show results

In [ ]:
# Use random.seed to get comparable results 
np.random.seed(42)

# Create datasets
# We use the iris dataset, split into two-dimensional subsets
iris = dsets.load_iris()
iris2d_1 = (iris.data[:,(0,1)], iris.target)
iris2d_2 = (iris.data[:,(0,2)], iris.target)
iris2d_3 = (iris.data[:,(1,2)], iris.target)
iris2d_4 = (iris.data[:,(1,3)], iris.target)
iris2d_5 = (iris.data[:,(2,3)], iris.target)

# Prepare the result figure
plt.figure(figsize=(25,25))
#plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05, hspace=.01)
plot_num = 1

# Set default parameters
default_params = {'n_clusters': 3,
                  'iterations': 3,
                  'eps': 1. }

# Set datasets
datasets = [
    iris2d_1,
    iris2d_2,
    iris2d_3,
    iris2d_4,
    iris2d_5,
    ]

# Set clustering algorithms
clustering_algorithms = (
    ('DP-Clustering', dp_clustering, {'c_bound': .1}),
    ###### TODO (Part 2) #######
    # Also use ExpQ-Clustering #
    # Simply uncomment this line: #
    # ('ExpQ-Clustering', expq_clustering, {}),
)

# For all datasets...
for i_dataset, dataset in enumerate(datasets):
    params = default_params.copy()

    # Prepare the current dataset
    X, _ = dataset
    X = StandardScaler().fit_transform(X)

    # From all data points: Choose fixed initial centroids to get comparable results for different algorithms
    idx = np.random.choice(len(X), params["n_clusters"], replace=False)
    init_centroids = copy.deepcopy(X[idx, :])

    # For all clustering algorithms...
    for name, algorithm, algo_params in clustering_algorithms:
        if algo_params:
            y_pred, centroids, avg_distances = algorithm(X, params['n_clusters'], params['iterations'], params['eps'], init_centroids, algo_params)
        else:
            y_pred, centroids, avg_distances = algorithm(X, params['n_clusters'], params['iterations'], params['eps'], init_centroids)
        
        # Create new subplot
        plt.subplot(len(datasets), len(clustering_algorithms)*2, plot_num)
        if i_dataset == 0:
            plt.title(f"{name} (result)", size=18)
        
        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))

        # Plot the resulting clustering
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])
        plt.scatter(centroids[:,0], centroids[:,1], c="k", marker="x", label="centroid")
        plt.xticks(())
        plt.yticks(())

        plot_num += 1

        plt.subplot(len(datasets), len(clustering_algorithms)*2, plot_num)
        if i_dataset == 0:
            plt.title(f"{name} (performance)", size=18)
        plt.plot(avg_distances)
        plt.xticks(range(params['iterations']))
        plt.xlabel("iteration")
        plt.ylabel("avg_min_distance")

        plot_num += 1

plt.show()